## Importacion de modulos necesarios

In [4]:
import simpy
import random
import pandas as pd
import matplotlib.pyplot as plt

## Configuracion de variables necesarias

In [152]:
CLIENTS = ['Carlos', 'Raul', 'Alejandro', 'Sofia', 'Laura', 'Juan', 'Jose', 'Pedro', 'Sebastian', 'Maria', 'Aitana', 'Dana', 'Betzie', 'Monica', 'Rebecca', 'Pepe', 'Lucas', 'Rubio']
RESTAURANTS = ['Restaurante-1', 'Restaurante-2', 'Restaurante-3', 'Restaurante-4', 'Restaurante 5']
DELIVERY_MEN = ['Daniela', 'David', 'Mario', 'Roberto', 'Rony', 'Alberto', 'Joaquin']

TOTAL_DELIVERY_MEN = len(DELIVERY_MEN)
TOTAL_CLIENTS = len(CLIENTS)
TOTAL_RESTAURANTS = len(RESTAURANTS)

## Helpers

In [155]:
def delivery_availability(env, delivery_men):
    while True:
        # Tiempo aleatorio de espera para disponibilidad de repartidor
        yield env.timeout(random.randint(20, 60)) # 20min y 1h

## Codigo principal

In [258]:
class Order:
    def __init__(self, env, id, client, restaurant):
        self.env = env
        self.client = client
        self.id = id
        self.restaurant = restaurant
        self.preparation_time = round(15 / self.restaurant.num_of_chefs)
        self.delivery_time = random.randint(10, 20)
        self.max_waiting_time = random.randint(15, 40)

In [260]:
class Client:
    def __init__(self, id, name):
        self.id = id
        self.name = name

In [262]:
class Delivery:
    def __init__(self, id, name):
        self.id = id
        self.name = name

In [264]:
class Restaurant:
    def __init__(self, env, id, name, num_of_cashiers, num_of_chefs):
        self.id = id
        self.name = name
        self.num_of_cashiers = num_of_cashiers
        self.env = env
        self.num_of_chefs = num_of_chefs
        self.cashiers = simpy.Resource(self.env, capacity=num_of_cashiers)
        self.chefs = simpy.Resource(self.env, capacity=num_of_chefs)

In [266]:
class Main: 
    def __init__(self, env, clients, restaurants, delivery_men):
        self.env = env
        self.clients = clients
        self.restaurant = restaurants
        self.delivery_men = delivery_men
        self.list_of_busy_delivery_men = [] 

    def order_arrival(self, interval):
        client_id = 0
        while True:
            yield self.env.timeout(random.expovariate(1.0 / interval))
            client = random.choice(self.clients)
            restaurant = random.choice(self.restaurant)
            client_id += 1
            print(f'{round(self.env.now)}: Nuevo pedido realizado por: {client.name} en el restaurante: {restaurant.name}')
            self.env.process(self.process_order(client, restaurant))
    
    def process_order(self, client, restaurant):
        initial_time = self.env.now
        order = Order(self.env, random.sample(list(range(1, 1000000)), 1)[0], client, restaurant)
        time_arrives_order = self.env.now

        with self.delivery_men.request() as delivery_req:
            yield delivery_req
            global df_events  
            global df_restaurant_data
            
            # [Daniela, Mario, David] --> Todos los deliveries
            # [Daniela] -- Deliveries ocupados
            delivery = random.choice(list(set(delivery_arr) - set(self.list_of_busy_delivery_men)))
            self.list_of_busy_delivery_men.append(delivery)
            
            print(f'{round(self.env.now)}: Delivery {delivery.name} toma la orden del cliente: {client.name} y va hacia el restaurante: {restaurant.name}')
            time_order_taken = round(self.env.now - time_arrives_order)
            time_afer_take_order = self.env.now

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(self.env.now),
                'delivery': delivery.name,
                'time_per_event': time_order_taken,
                'description_event': 'Orden tomada por delivery'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])

            df_events = pd.concat([df_events, df_temp_event])

            # Tiempo en llegar al restaurante
            yield self.env.timeout(random.randint(2, 7))
            time_to_restaurant = round(self.env.now - time_afer_take_order)

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(self.env.now),
                'delivery': delivery.name,
                'time_per_event': time_to_restaurant,
                'description_event': 'En camino al restaurante'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])

            df_events = pd.concat([df_events, df_temp_event])
                
            with restaurant.cashiers.request() as cashier_req:
                yield cashier_req
                print(f'{round(self.env.now)}: Pedido del cliente {client.name} esta siendo atendido en caja en el restaurante: {restaurant.name}')
                time_at_cashier = round(self.env.now)
                
                # Tiempo de espera en caja
                yield self.env.timeout(random.randint(1, 3))
                time_after_cashier = round(self.env.now - time_at_cashier)

                df_temp_event = pd.DataFrame({
                    'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                    'order': order.id,
                    'order_client_id': order.client.id,
                    'order_client_name': order.client.name,
                    'time': round(time_at_cashier),
                    'delivery': delivery.name,
                    'time_per_event': time_after_cashier,
                    'description_event': 'En caja'
                }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
                df_events = pd.concat([df_events, df_temp_event])
            
            with restaurant.chefs.request() as chef_req:
                yield chef_req
                print(f'{round(self.env.now)}: Pedido del cliente {client.name} en preparacion en el restaurante: {restaurant.name}')
                time_start_waiting_order = round(self.env.now)
                
                yield self.env.timeout(order.preparation_time)
                time_after_waiting_order = round(self.env.now) - time_start_waiting_order
                print(f'{round(self.env.now)}: Orden del cliente {client.name} entregada al repartidor {delivery.name} en el restaurante: {restaurant.name}')

                df_temp_event = pd.DataFrame({
                    'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                    'order': order.id,
                    'order_client_id': order.client.id,
                    'order_client_name': order.client.name,
                    'time': round(time_start_waiting_order),
                    'delivery': delivery.name,
                    'time_per_event': time_after_waiting_order,
                    'description_event': 'Esperando orden'
                }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
                df_events = pd.concat([df_events, df_temp_event])

                df_temp_restaurant_data = pd.DataFrame({
                    'id': restaurant.id,
                    'name': restaurant.name,
                    'order': order.id,
                    'num_of_cashiers': restaurant.num_of_cashiers,
                    'num_of_chefs': restaurant.num_of_chefs,
                    'time_in_cashier': time_after_cashier,
                    'time_in_kitchen': time_after_waiting_order,
                    'total_time_in_restaurant': time_after_cashier + time_after_waiting_order
                }, index=[0 if df_restaurant_data.empty else df_restaurant_data.index[-1] + 1])

                df_restaurant_data = pd.concat([df_restaurant_data, df_temp_restaurant_data])
                
                time_go_to_client = self.env.now
            
            yield self.env.timeout(order.delivery_time)
            total_time = self.env.now - initial_time
            time_order_delivered = round(self.env.now - time_go_to_client)
            print(f'{round(self.env.now)}: Pedido del cliente: {client.name} entregado por: {delivery.name} en un total de: {round(total_time)}min. Estimado de entrega: {order.max_waiting_time}min')

            df_temp_event = pd.DataFrame({
                'id': 1 if df_events.empty else df_events.index[-1] + 1, 
                'order': order.id,
                'order_client_id': order.client.id,
                'order_client_name': order.client.name,
                'time': round(time_go_to_client),
                'delivery': delivery.name,
                'time_per_event': time_order_delivered,
                'description_event': 'Orden entregada'
            }, index=[0 if df_events.empty else df_events.index[-1] + 1])
        
            df_events = pd.concat([df_events, df_temp_event])
            
            self.list_of_busy_delivery_men.remove(delivery)

            global df_order  
            df_temp_order = pd.DataFrame({
                'id': order.id,
                'time': round(initial_time),
                'client': client.name,
                'restaurant': restaurant.name,
                'delivery': delivery.name,
                'estimated_time': order.max_waiting_time,
                'real_time_deliver': round(total_time),
                'state': 'A tiempo' if round(total_time) <= order.max_waiting_time else 'Fuera de tiempo'
            }, index=[0 if df_order.empty else df_order.index[-1] + 1])

            df_order = pd.concat([df_order, df_temp_order])
            print('-' * 100)

In [268]:
env = simpy.Environment()

# VARIABLES NECESARIAS
delivery_men = simpy.Resource(env, capacity=TOTAL_DELIVERY_MEN)
clients = [Client(i, name) for i, name in enumerate(CLIENTS)]
restaurants = [Restaurant(env, i, name, random.randint(1, 3), random.randint(2, 5)) for i, name in enumerate(RESTAURANTS)]
delivery_arr = [Delivery(i, name) for i, name in enumerate(DELIVERY_MEN)]

# Inicializacion de dataframes
clients_data = [{'id': client.id, 'name': client.name} for client in clients]
df_client = pd.DataFrame(clients_data)

restaurants_data = [{'id': restaurant.id, 'name': restaurant.name, 'num_of_cashiers': restaurant.num_of_cashiers, 'num_of_chefs': restaurant.num_of_chefs} for restaurant in restaurants]
df_restaurant = pd.DataFrame(restaurants_data)

delivery_data = [{'id': delivery.id, 'name': delivery.name} for delivery in delivery_arr]
df_delivery = pd.DataFrame(delivery_data)

df_order = pd.DataFrame(columns=['id', 'time', 'client', 'restaurant', 'delivery', 'estimated_time', 'real_time_deliver', 'state'])
df_restaurant_data = pd.DataFrame(columns=['id', 'name', 'order', 'num_of_cashiers', 'num_of_chefs', 'time_in_cashier', 'time_in_kitchen', 'total_time_in_restaurant'])
df_events = pd.DataFrame(columns=['id', 'order', 'order_client_id', 'order_client_name', 'time', 'delivery', 'time_per_event', 'description_event'])

# instancia de la clase main
app = Main(env, clients, restaurants, delivery_men)

# PROCESOS
env.process(app.order_arrival(10))
env.process(delivery_availability(env, delivery_men))

# Ejecutar la simulacion
env.run(5000)

9: Nuevo pedido realizado por: Juan en el restaurante: Restaurante 5
9: Delivery David toma la orden del cliente: Juan y va hacia el restaurante: Restaurante 5
9: Nuevo pedido realizado por: Sebastian en el restaurante: Restaurante-1
9: Delivery Roberto toma la orden del cliente: Sebastian y va hacia el restaurante: Restaurante-1
9: Nuevo pedido realizado por: Juan en el restaurante: Restaurante-2
9: Delivery Joaquin toma la orden del cliente: Juan y va hacia el restaurante: Restaurante-2
11: Pedido del cliente Sebastian esta siendo atendido en caja en el restaurante: Restaurante-1
14: Pedido del cliente Juan esta siendo atendido en caja en el restaurante: Restaurante 5
14: Pedido del cliente Sebastian en preparacion en el restaurante: Restaurante-1
14: Pedido del cliente Juan esta siendo atendido en caja en el restaurante: Restaurante-2
15: Pedido del cliente Juan en preparacion en el restaurante: Restaurante-2
16: Pedido del cliente Juan en preparacion en el restaurante: Restaurante 

In [270]:
df_restaurant_data

,id,name,order,num_of_cashiers,num_of_chefs,time_in_cashier,time_in_kitchen,total_time_in_restaurant
0,0,Restaurante-1,229798,3,4,3,4,7
1,1,Restaurante-2,811551,1,4,1,4,5
2,4,Restaurante 5,972055,1,4,2,4,6
3,3,Restaurante-4,826678,3,4,1,4,5
4,1,Restaurante-2,255593,1,4,1,4,5
...,...,...,...,...,...,...,...,...
499,3,Restaurante-4,82611,3,4,1,4,5
500,3,Restaurante-4,441381,3,4,3,4,7
501,3,Restaurante-4,514088,3,4,3,4,7
502,1,Restaurante-2,759501,1,4,1,4,5


In [272]:
df_order

,id,time,client,restaurant,delivery,estimated_time,real_time_deliver,state
0,229798,9,Sebastian,Restaurante-1,Roberto,40,26,A tiempo
1,826678,17,Monica,Restaurante-4,Daniela,15,20,Fuera de tiempo
2,972055,9,Juan,Restaurante 5,David,34,29,A tiempo
3,811551,9,Juan,Restaurante-2,Joaquin,31,30,A tiempo
4,255593,20,Maria,Restaurante-2,Alberto,22,25,Fuera de tiempo
...,...,...,...,...,...,...,...,...
498,667942,4941,Maria,Restaurante 5,David,19,24,Fuera de tiempo
499,82611,4944,Juan,Restaurante-4,Mario,27,25,A tiempo
500,441381,4944,Dana,Restaurante-4,Alberto,19,30,Fuera de tiempo
501,514088,4955,Maria,Restaurante-4,Daniela,35,30,A tiempo


In [274]:
df_events

,id,order,order_client_id,order_client_name,time,delivery,time_per_event,description_event
0,1,972055,5,Juan,9,David,0,Orden tomada por delivery
1,1,229798,8,Sebastian,9,Roberto,0,Orden tomada por delivery
2,2,811551,5,Juan,9,Joaquin,0,Orden tomada por delivery
3,3,229798,8,Sebastian,11,Roberto,2,En camino al restaurante
4,4,972055,5,Juan,14,David,5,En camino al restaurante
...,...,...,...,...,...,...,...,...
2515,2515,503794,8,Sebastian,4992,Daniela,4,En camino al restaurante
2516,2516,503794,8,Sebastian,4992,Daniela,1,En caja
2517,2517,450500,13,Monica,4995,Alberto,0,Orden tomada por delivery
2518,2518,759501,14,Rebecca,4985,Roberto,10,Orden entregada


## 1. Analisis entregas a tiempo y fuera de tiempo por Delivery
**¿Cuantas entregas se relizaron a tiempo y cuantas se realizaron fuera de tiempo?** (Esto mediria que tan bueno es el rendimiento de un repartidor)

In [277]:
df_order.groupby(['delivery', 'state']).size().unstack(fill_value=0)

state,A tiempo,Fuera de tiempo
delivery,,
Alberto,42,26
Daniela,48,27
David,44,27
Joaquin,46,25
Mario,47,22
Roberto,49,31
Rony,44,25


## 2. Analisis de Cuantas ordenes pidio cada cliente ordenadas del cliente que mas pidio al que menos pidio
**¿Quienes fueron los clientes que mas ordenes realizaron?** (Esto serviria para poder observar la fidelidad de los clientes)

In [280]:
df_order.groupby(['client']).size().sort_values(ascending=False).reset_index(name='Number of orders')

,client,Number of orders
0,Dana,38
1,Maria,35
2,Raul,33
3,Carlos,33
4,Sebastian,31
5,Pepe,30
6,Pedro,29
7,Rebecca,28
8,Lucas,28
9,Juan,27


## 3. Analisis de cantidad de ordenes atendidas por restaurante
**¿Cuantas ordenes fueron atendidas por restaurante y cuales restaurantes atendieron mas ordenes?** (Esto serviria para medir el rendimiento de cada restaurante)

In [283]:
df_order.groupby(['restaurant']).size().sort_values(ascending=False).reset_index(name='Number of orders')

,restaurant,Number of orders
0,Restaurante-1,120
1,Restaurante 5,107
2,Restaurante-2,96
3,Restaurante-3,96
4,Restaurante-4,84


## 4. Analis de tiempo promedio de entrega por delivery
**¿Cual es el tiempo promedio total de un pedido por delivery?** (Esto ayudaria a poder observar la velocidad con la que el repartidor realiza las entregas)

In [286]:
df_order.groupby('delivery')['real_time_deliver'].mean().reset_index().sort_values(by='real_time_deliver', ascending=False)

,delivery,real_time_deliver
6,Rony,26.347826
3,Joaquin,25.915493
5,Roberto,25.8
0,Alberto,25.75
1,Daniela,25.493333
2,David,25.380282
4,Mario,25.318841


## 5. Analisis de una orden en especifico
**Ver el log (Historial de tiempo cronologico) de una orden en especifico por evento** (Esto sirve para llevar un log/historial cronologico del tiempo de una orden en especifico por evento)

In [289]:
order_id = 3893
df_analysis_by_order = df_events[df_events['order'] == order_id]
df_analysis_by_order.sort_values('time')
df_analysis_by_order[['time', 'time_per_event', 'description_event']]

,time,time_per_event,description_event


## 6. Analis del tiempo total tomado por orden/pedido
**¿Cual es el tiempo total tomado de entrega por orden?** (Esto sirve para ver cual es el tiempo total en minutos que tardo una orden en ser entregada)

In [292]:
df_events.groupby('order')['time_per_event'].sum().reset_index()

,order,time_per_event
0,674,27
1,3238,27
2,6098,20
3,12115,20
4,13231,21
...,...,...
500,983267,25
501,986911,22
502,987755,29
503,993376,30


## 7. Analisis de la media de los eventos que mas tiempo toman
**¿Cual es la media del tiempo por evento?** (Esto sirve para poder observar cual es el evento que mas tiempo toma y poder mejorar el rendimiento en ese aspecto en especifico)

In [295]:
df_events.groupby('description_event')['time_per_event'].mean().sort_values(ascending=False).reset_index()

,description_event,time_per_event
0,Orden entregada,14.84493
1,En camino al restaurante,4.420635
2,Esperando orden,4.190476
3,En caja,2.041667
4,Orden tomada por delivery,0.188119


## 8. Analisis tiempo real vs tiempo estimado
**¿Hubo rendimiento positivo entre el tiempo real de entrega con respecto al tiempo estimado? (En caso de ser negativo quiere decir que en total, la media del tiempo real de entregas fue bajo con respecto al tiempo estimado de entregas)** (Esto sirve para ver la media de la diferencia entre tiempo real y tiempo estimado)

In [298]:
df_order['time_difference'] = df_order['real_time_deliver'] - df_order['estimated_time']
df_order['time_difference'].mean()

-2.2306163021868786

## 9. Analisis de la media de la diferencia de tiempo entre tiempo real de entrega y tiempo estimado de entrega
**¿Cual fue el rendimiento de cada repartidor con respecto al tiempo estimado y tiempo real de entrega?** (Esto ayuda a ver si la media del rendimiento por tiempo real y tiempo estimado por repartidor es positivo o no)

In [301]:
df_order.groupby('delivery')['time_difference'].mean().reset_index().sort_values(by='time_difference', ascending=False)

,delivery,time_difference
0,Alberto,-1.279412
5,Roberto,-1.85
6,Rony,-1.956522
3,Joaquin,-2.225352
2,David,-2.492958
1,Daniela,-2.786667
4,Mario,-3.014493


## 10. Analisis de la media del tiempo total de atencion por restaurante
**¿Cuales fueron los restaurantes que mejor promedio en tiempo de atencion tuvieron?** (Medir rendimiento por restaurante en terminos de tiempo total en ser atendido)

In [304]:
df_restaurant_data.groupby('name')['total_time_in_restaurant'].mean().reset_index().sort_values(by='total_time_in_restaurant', ascending=False)

,name,total_time_in_restaurant
3,Restaurante-3,7.0
0,Restaurante 5,6.168224
1,Restaurante-1,6.099174
4,Restaurante-4,5.988095
2,Restaurante-2,5.916667
